# SelectGrid

Extending [StyleGrid](./StyleGrid%20I.ipynb) even further, `SelectGrid` exposes the interactive state of the grid back to a kernel.

In [ ]:
from wxyz.datagrid import SelectGrid, GridStyle

In [ ]:
import numpy as np, pandas as pd, traitlets as T, ipywidgets as W, importnb
from wxyz.html import AlphaColorPicker
from wxyz.lab import DockBox

In [ ]:
flex = dict(layout=dict(flex="1"))

In [ ]:
with importnb.Notebook():
    try:
        from StyleGrid_I import make_a_grid_styler
    except:
        from .StyleGrid_I import make_a_grid_styler

## Selection Style 

`GridStyle` exposes all of the knobs, but here we're just interested in the selection values:

In [ ]:
def make_select_grid(GridClass=SelectGrid, df=None):
    pg = DockBox(layout=dict(height="60vh"))
    dg = pg.dg = GridClass()
    dg2 = pg.dg2 = GridClass()
    dg3 = pg.dg3 = GridClass()
    
    rando = None
    if df is not None:
        dg.value = df
    else:
        with W.Output():
            @W.interact
            def rando(rows=(1, 1000), cols=(1, 1000)):
                dg.value = np.random.rand(rows, cols)
    x, y = [
        [
            W.jslink((w, "value"), (dg, f"scroll_{v}")),
            W.jslink((pg.dg, f"max_{v}"), (w, "max")),
            w,
        ][-1]
        for i, v in enumerate("xy")
        for w in [W.IntSlider(description=f"${v}$", layout=dict(flex="1"))] 
    ]
    
    def gloc(g):
        if g.selections:
            s = g.selections[0]
            return g.value.iloc[s[2]: s[3]+ 1, s[0]:s[1] + 1]
    
    def _slice(c):
        if dg.selections:
            s = dg.selections[0]
            dg2.value = gloc(dg)
            dg.description = f"{dg2.value.values.sum()}"
        
    
    def _slice2(c):
        if dg2.selections:
            s = dg2.selections[0]
            dg3.value = gloc(dg2)
            dg2.description = f"{dg3.value.values.sum()}"
    
    def _slice3(c):
        if dg3.selections:
            s = dg3.selections[0]
            v = gloc(dg3)
            dg3.description = f"{v.values.sum()}"
    
    dg.observe(_slice, ["selections", "value"])
    dg2.observe(_slice2, ["selections", "value"])
    dg3.observe(_slice3, ["selections", "value"])
    
    tabs = W.ToggleButton(description="tabs")
    T.link((tabs, "value"), (pg, "hide_tabs"))
    
    vp = W.Text(description="view")
    T.dlink((pg.dg, "viewport"), (vp, "value"), str)
    
    c = make_a_grid_styler(None, pg.dg, pattern="select", default_color="rgba(255,0,255,0.125)")
    
    hv = W.Text(description="hover")
    T.dlink((pg.dg, "hover_row"), (hv, "value"), lambda x: f"({pg.dg.hover_row}, {pg.dg.hover_column})")
    T.dlink((pg.dg, "hover_column"), (hv, "value"), lambda y: f"({pg.dg.hover_row}, {pg.dg.hover_column})")
    
    pg.children = [
        W.VBox([*(rando.widget.children if rando else []), vp, hv, x, y, tabs, c]), 
        dg, 
        dg2,
        dg3
    ]
    pg.dock_layout ={
     'type': 'split-area',
     'orientation': 'horizontal',
     'children': [{'type': 'tab-area', 'widgets': [1], 'currentIndex': 0},
      {'type': 'tab-area', 'widgets': [2], 'currentIndex': 0},
      {'type': 'split-area',
       'orientation': 'vertical',
       'children': [{'type': 'tab-area', 'widgets': [3], 'currentIndex': 0},
        {'type': 'tab-area', 'widgets': [0], 'currentIndex': 0}],
       'sizes': [1, 1]}],
     'sizes': [4, 2, 2]
    }
    return pg

In [ ]:
if __name__ == "__main__":
    pg = make_select_grid()
    display(pg)

    pg.dg.viewport = (21, 30, 10, 18)

    i_0 = W.IntSlider(description="$i_0$")
    T.dlink((i_0, "value"), (pg.dg, "viewport"), lambda vc: (vc, *pg.dg.viewport[1:]))

    j_0 = W.IntSlider(description="$j_0$")
    T.dlink((j_0, "value"), (pg.dg, "viewport"), lambda vr: (*pg.dg.viewport[:2], vr, *pg.dg.viewport[3:]))
    display(W.HBox([i_0, j_0]))

In [ ]:
if __name__ == "__main__":
    with __import__("importnb").Notebook():
        from wxyz.notebooks import Utils
        Utils.maybe_log_widget_counts()